In [280]:
import numpy as np
import pandas as pd
import random
from enum import Enum
from datetime import datetime, timedelta

In [281]:
class Currency(Enum):
  USD = ("USD", 1.0)         # United States Dollar (USD per USD)
  AED = ("AED", 0.27)        # UAE Dirham (AED per USD)
  INR = ("INR", 83.31)       # Indian Rupee (INR per USD)
  JPY = ("JPY", 151.62)      # Japanese Yen (JPY per USD)
  CNY = ("CNY", 7.23)        # Chinese Yuan (CNY per USD)
  SGD = ("SGD", 1.35)        # Singapore Dollar (SGD per USD)
  AUD = ("AUD", 1.52)        # Australian Dollar (AUD per USD)
  CAD = ("CAD", 1.36)        # Canadian Dollar (CAD per USD)
  MXN = ("MXN", 16.46)       # Mexican Peso (MXN per USD)
  EUR = ("EUR", 0.92)        # Euro (EUR per USD)
  GBP = ("GBP", 0.79)        # British Pound Sterling (GBP per USD)
  CHF = ("CHF", 0.90)        # Swiss Franc (CHF per USD)
  SEK = ("SEK", 10.67)       # Swedish Krona (SEK per USD)

In [282]:
def inr_to_currency(inr_amount, to_currency):
  usd_amount = inr_amount / Currency.INR.value[1]
  converted_amount = usd_amount * to_currency.value[1]
  return converted_amount


inr_to_currency(1000, Currency.USD)
inr_to_currency(1000, Currency.EUR)
inr_to_currency(1000, Currency.JPY)

12.003360941063498

11.043092065778419

1819.9495858840476

In [283]:
class City(Enum):
  PUNE = "Pune"
  TOKYO = "Tokyo"
  DUBAI = "Dubai"
  SYDNEY = "Sydney"
  SHANGHAI = "Shanghai"
  SINGAPORE = "Singapore"
  NEW_DELHI = "New Delhi"
  MUMBAI = "Mumbai"
  NEW_YORK = "New York"
  LOS_ANGELES = "Los Angeles"
  LONDON = "London"
  PARIS = "Paris"
  BERLIN = "Berlin"


CityCurrency = {
    City.PUNE: Currency.INR,
    City.TOKYO: Currency.JPY,
    City.DUBAI: Currency.AED,
    City.SYDNEY: Currency.AUD,
    City.SHANGHAI: Currency.CNY,
    City.SINGAPORE: Currency.SGD,
    City.NEW_DELHI: Currency.INR,
    City.MUMBAI: Currency.INR,
    City.NEW_YORK: Currency.USD,
    City.LOS_ANGELES: Currency.USD,
    City.LONDON: Currency.GBP,
    City.PARIS: Currency.EUR,
    City.BERLIN: Currency.EUR,
}

In [284]:
class Amount(Enum):
  INR_1k = 1000
  INR_5k = 5000
  INR_10k = 10_000
  INR_50k = 50_000
  INR_100k = 100_000
  INR_500k = 500_000
  INR_1m = 1_000_000
  INR_5m = 5_000_000
  INR_10m = 10_000_000


class AmountRange(Enum):
  LOW = (Amount.INR_1k, Amount.INR_5k, Amount.INR_10k)
  MEDIUM = (Amount.INR_10k, Amount.INR_50k, Amount.INR_100k)
  HIGH = (Amount.INR_100k, Amount.INR_500k, Amount.INR_1m)
  MAX = (Amount.INR_1m, Amount.INR_5m, Amount.INR_10m, None)


def get_eligible_amount_ranges(amount_range):
  amount_ranges = list(AmountRange)
  max_index = amount_ranges.index(amount_range) + 1
  eligible_ranges = amount_ranges[:max_index]
  return eligible_ranges


def compare_amount_range(service_range, my_range):
  eligible_ranges = get_eligible_amount_ranges(my_range)
  if service_range in eligible_ranges:
    return True
  else:
    return False


def get_random_amount_within_range(amount_range: AmountRange):
  return random.choice(amount_range.value)

In [285]:
class TransactionType(Enum):
  WITHDRAWAL = "Withdrawal"  # Money withdrawn from an account
  DEPOSIT = "Deposit"        # Money deposited into an account
  TRANSFER = "Transfer"      # Money transferred from one account to another
  PAYMENT = "Payment"        # Payment made for goods or services

In [328]:
class TransactionChannel(Enum):
  ATM = "ATM"                        # Cash withdrawals
  ONLINE_BANKING = "Online Banking"  # Banking via web
  UPI = "UPI"                        # Real-time payment system (India)
  POS = "POS"                        # Retail transactions
  BANK = "Bank"                      # Generic bank transactions


TransactionChannelMaxAmount = {
    TransactionChannel.ATM: AmountRange.MEDIUM,
    TransactionChannel.ONLINE_BANKING: AmountRange.HIGH,
    TransactionChannel.UPI: AmountRange.MEDIUM,
    TransactionChannel.POS: AmountRange.MEDIUM,
    TransactionChannel.BANK: AmountRange.MAX,
}

EligibleChannelsForTransaction = {
    TransactionType.WITHDRAWAL: [TransactionChannel.ATM, TransactionChannel.BANK],
    TransactionType.DEPOSIT: [TransactionChannel.ATM, TransactionChannel.BANK],
    TransactionType.TRANSFER: [TransactionChannel.ATM, TransactionChannel.ONLINE_BANKING, TransactionChannel.BANK, TransactionChannel.UPI],
    TransactionType.PAYMENT: [
        TransactionChannel.BANK, TransactionChannel.ONLINE_BANKING, TransactionChannel.UPI, TransactionChannel.POS
    ]
}

cities_with_upi_support = [City.SINGAPORE, City.DUBAI,
                           City.PARIS, City.LONDON, City.NEW_DELHI, City.MUMBAI, City.PUNE]


def get_eligible_channels(transaction_type, amount_range, city):
  channels = EligibleChannelsForTransaction[transaction_type]
  eligible_channels = [channel for channel in channels
                       if compare_amount_range(amount_range, TransactionChannelMaxAmount[channel])]

  if city not in cities_with_upi_support:
    eligible_channels = [
        channel for channel in eligible_channels if channel != TransactionChannel.UPI]

  return eligible_channels

In [287]:
class CardType(Enum):
  CREDIT_CARD = "Credit Card"    # Borrow funds for purchases
  DEBIT_CARD = "Debit Card"      # Deducts money from checking account
  PREPAID_CARD = "Prepaid Card"  # Pre-loaded funds for purchases
  NA = "NA"                      # Not Applicable/Available


CardTypeMaxAmount = {
    CardType.CREDIT_CARD: AmountRange.MEDIUM,
    CardType.DEBIT_CARD: AmountRange.MEDIUM,
    CardType.PREPAID_CARD: AmountRange.LOW,
    CardType.NA: None,
}


def get_eligible_cards(amount_range: AmountRange):
  eligible_cards = [key for key, value in CardTypeMaxAmount.items()
                    if key is not CardType.NA and compare_amount_range(amount_range, value)]
  return eligible_cards

In [334]:
from itertools import chain


class PaymentApp(Enum):
  WHATSAPP_PAY = "WhatsApp Pay"  # Transaction via WhatsApp Pay
  PAYTM = "Paytm"                # Transaction via Paytm
  PHONEPE = "PhonePe"            # Transaction via PhonePe
  GOOGLE_PAY = "Google Pay"      # Transaction via Google Pay
  ALIPAY = "Alipay"              # Transaction via Alipay (China)
  WECHAT_PAY = "WeChat Pay"      # Transaction via WeChat Pay (China)
  PAYPAL = "PayPal"              # Transaction via PayPal
  NA = "NA"                      # Not Applicable/Available


class PaymentAppCategory(Enum):
  CHINESE_APPS = [PaymentApp.ALIPAY, PaymentApp.WECHAT_PAY]
  UPI_APPS = [PaymentApp.WHATSAPP_PAY, PaymentApp.PHONEPE, PaymentApp.PAYTM]
  GLOBAL_APPS = [PaymentApp.GOOGLE_PAY, PaymentApp.PAYPAL]


CityPaymentApp = {
    City.PUNE: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.TOKYO: [PaymentAppCategory.CHINESE_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.DUBAI: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.SYDNEY: [PaymentAppCategory.GLOBAL_APPS],
    City.SHANGHAI: [PaymentAppCategory.CHINESE_APPS,
                    PaymentAppCategory.GLOBAL_APPS],
    City.SINGAPORE: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.NEW_DELHI: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.MUMBAI: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.NEW_YORK: [PaymentAppCategory.GLOBAL_APPS],
    City.LOS_ANGELES: [PaymentAppCategory.GLOBAL_APPS],
    City.LONDON: [PaymentAppCategory.GLOBAL_APPS],
    City.PARIS: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
    City.BERLIN: [PaymentAppCategory.UPI_APPS, PaymentAppCategory.GLOBAL_APPS],
}


for key in CityPaymentApp:
  CityPaymentApp[key] = list(chain.from_iterable(
      [payment_app_category.value for payment_app_category in CityPaymentApp[key]]))

In [289]:
class MerchantCategory(Enum):
  RETAIL = "Retail"                       # Physical goods
  ONLINE_SERVICES = "Online Services"     # Online services
  ECOMMERCE = "E-commerce"                # Online marketplaces
  DINING = "Dining"                       # Restaurants and cafes
  ENTERTAINMENT = "Entertainment"         # Venues for entertainment
  TRANSPORT = "Transport"                 # Transportation services
  HEALTHCARE = "Healthcare"               # Healthcare services
  TRAVEL = "Travel"                       # Travel agencies
  EDUCATION = "Education"                 # Educational services
  FINANCIAL = "Financial"                 # Financial institutions
  REAL_ESTATE = "Real Estate"             # Real estate services
  NA = "NA"                               # Not Applicable/Available
  OTHERS = "Others"                       # Other merchant categories


MerchantAmountRange = {
    MerchantCategory.RETAIL: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.ONLINE_SERVICES: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.ECOMMERCE: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.DINING: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.ENTERTAINMENT: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.TRANSPORT: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.TRAVEL: (AmountRange.LOW, AmountRange.MEDIUM),
    MerchantCategory.HEALTHCARE: (AmountRange.LOW, AmountRange.HIGH),
    MerchantCategory.EDUCATION: (AmountRange.LOW, AmountRange.HIGH),
    MerchantCategory.FINANCIAL: (AmountRange.MEDIUM, AmountRange.MAX),
    MerchantCategory.REAL_ESTATE: (AmountRange.HIGH, AmountRange.MAX),
    MerchantCategory.NA: (None, None),
    MerchantCategory.OTHERS: (None, None),
}


def get_eligible_merchants(amount_range: AmountRange):
  eligible_merchants = [key for key, value in MerchantAmountRange.items()
                        if key not in (MerchantCategory.NA, MerchantCategory.OTHERS) and compare_amount_range(value[0], amount_range)]
  return eligible_merchants

In [290]:
df = pd.DataFrame({
    'timestamp': [],             # Date and time of transaction
    'transaction_id': [],        # Unique identifier for each transaction
    'account_id': [],            # Unique identifier for the bank account
    'amount': [],                # Monetary value of the transaction
    'currency': [],              # Currency of the transaction
    'city': [],                  # City where the transaction occurred
    'transaction_type': [],      # Nature or category of the transaction
    'transaction_channel': [],   # Channel through which the transaction was made
    'card_type': [],             # Type of card used for the transaction
    'payment_app': [],           # Payment app used for the transaction
    'merchant_id': [],           # Unique identifier for the merchant
    'merchant_category': [],     # Category of the merchant
})

In [291]:
def stochastic_shift(base, variance):
  return base + base * random.uniform(-variance, +variance)


stochastic_shift(100, 0.1)

91.93453963658277

In [292]:
def variance_split(total, num_parts, variance):
  base_each = total / num_parts
  final_parts = [stochastic_shift(base=base_each, variance=variance)
                 for _ in range(num_parts)]
  return final_parts


variance_split(1000, 10, 0.1)

[97.20854446063466,
 105.32938073158753,
 109.38594137443175,
 96.60527823963142,
 108.06446590819509,
 97.04534757054222,
 90.51376786084471,
 108.01841689025355,
 94.31123523472036,
 91.6463533315771]

In [293]:
def print_object(obj):
  class_name = obj.__class__.__name__
  attributes = ", ".join(
      [f"{key}={value}" for key, value in obj.__dict__.items()])
  print(f"{class_name}({attributes})")

In [294]:
class CustomerProfile:
  def __init__(self, city, transaction_capacity):
    self.city = city
    self.transaction_capacity = transaction_capacity


customer_profiles = []
n = 50

for city in City:
  for amount_range in AmountRange:
    for _ in range(int(stochastic_shift(n, 0.1))):
      customer_profiles.append(CustomerProfile(city, amount_range))

len(City) * len(AmountRange) * n
len(customer_profiles)
print_object(customer_profiles[0])

2600

2579

CustomerProfile(city=City.PUNE, transaction_capacity=AmountRange.LOW)


In [318]:
def get_city_currency(city):
  return CityCurrency[city]


def get_transaction_types():
  return list(TransactionType)


def get_transaction_channels(amount_range):
  return get_eligible_channels(amount_range)


def get_card_types(amount_range):
  return get_eligible_cards(amount_range)


def get_payment_apps(city):
  return CityPaymentApp[city]


def get_random_weighted_element(lst):
  weights = list(reversed(range(1, len(lst) + 1)))
  return random.choices(lst, weights=weights)[0]

In [317]:
capable_amount_ranges = get_eligible_amount_ranges(AmountRange.MAX)

test = []

for _ in range(10**5):
  test.append(get_random_weighted_element(capable_amount_ranges))

test = np.array(test)

(test == AmountRange.LOW).sum()
(test == AmountRange.MEDIUM).sum()
(test == AmountRange.HIGH).sum()
(test == AmountRange.MAX).sum()

40023

29971

19879

10127

In [372]:
customer_profile = customer_profiles[0]
samples_to_generate = int(stochastic_shift(200, 0.5))
city = customer_profile.city
transaction_capacity = customer_profile.transaction_capacity
currency = get_city_currency(city)
transaction_types = get_transaction_types()
capable_amount_ranges = get_eligible_amount_ranges(transaction_capacity)

for i in range(samples_to_generate):
  capable_amount_range = get_random_weighted_element(capable_amount_ranges)

  transaction_type = get_random_weighted_element(
      list(reversed(transaction_types)))

  transaction_channel = random.choice(
      get_eligible_channels(transaction_type, capable_amount_range, city))

  payment_app = PaymentApp.NA
  card_type = CardType.NA
  eligible_merchant = MerchantCategory.NA

  if transaction_type == TransactionType.PAYMENT and transaction_channel is TransactionChannel.ONLINE_BANKING:
    if random.uniform(1, 100) > 20:
      payment_app = random.choice(CityPaymentApp[city])
    else:
      card_type = random.choice(get_eligible_cards(capable_amount_range))

  if transaction_type == TransactionType.PAYMENT and transaction_channel is TransactionChannel.UPI:
    payment_app = random.choice(CityPaymentApp[city])

  if transaction_type == TransactionType.PAYMENT and transaction_channel in (TransactionChannel.POS, TransactionChannel.ATM):
    card_type = random.choice(get_eligible_cards(capable_amount_range))

  if transaction_type == TransactionType.PAYMENT:
    eligible_merchant = random.choice(
        get_eligible_merchants(capable_amount_range))

  amount = get_random_amount_within_range(capable_amount_range)

  print(f"Iteration {i+1}:")
  print(f"City: {city}")
  print(f"Transaction Capacity: {transaction_capacity}")
  print(f"Currency: {currency}")
  print(f"Transaction Type: {transaction_type}")
  print(f"Transaction Channel: {transaction_channel}")
  print(f"Payment App: {payment_app}")
  print(f"Card Type: {card_type}")
  print(f"Eligible Merchant: {eligible_merchant}")
  print(f"Amount: {amount}")
  print("\n")

Iteration 1:
City: City.PUNE
Transaction Capacity: AmountRange.LOW
Currency: Currency.INR
Transaction Type: TransactionType.PAYMENT
Transaction Channel: TransactionChannel.UPI
Payment App: PaymentApp.PAYTM
Card Type: CardType.NA
Eligible Merchant: MerchantCategory.TRANSPORT
Amount: Amount.INR_10k


Iteration 2:
City: City.PUNE
Transaction Capacity: AmountRange.LOW
Currency: Currency.INR
Transaction Type: TransactionType.TRANSFER
Transaction Channel: TransactionChannel.ATM
Payment App: PaymentApp.NA
Card Type: CardType.NA
Eligible Merchant: MerchantCategory.NA
Amount: Amount.INR_1k


Iteration 3:
City: City.PUNE
Transaction Capacity: AmountRange.LOW
Currency: Currency.INR
Transaction Type: TransactionType.PAYMENT
Transaction Channel: TransactionChannel.POS
Payment App: PaymentApp.NA
Card Type: CardType.PREPAID_CARD
Eligible Merchant: MerchantCategory.ENTERTAINMENT
Amount: Amount.INR_1k


Iteration 4:
City: City.PUNE
Transaction Capacity: AmountRange.LOW
Currency: Currency.INR
Transactio